## Bentley auth using the client credentials flow

### Option 1
#### Automatic auth using the Evo Python SDK

In [ ]:
from evo.aio import AioTransport
from evo.oauth import ClientCredentialsAuthorizer, OAuthConnector

# Enter your client ID and client secret. The Evo scopes are handled automatically.
client_id = "<your-service-app-client-id>"
client_secret = "<your-service-app-client-secret>"

# Enter a user agent for your API requests.
user_agent = "Evo Demo app/1.0"

# Create an authorizer to obtain access tokens using the client credentials flow.
authorizer = ClientCredentialsAuthorizer(
    oauth_connector=OAuthConnector(
        transport=AioTransport(user_agent=user_agent),
        client_id=client_id,
        client_secret=client_secret,
    ),
)

# Print the default headers, including the Authorization header with the access token.
print(await authorizer.get_default_headers())

### Option 2
#### Manual auth using common Python packages

In [ ]:
import datetime
from http import HTTPStatus

import jwt
import requests

# Define your client ID, client secret and Evo scopes.
client_id = "<your-service-app-client-id>"
client_secret = "<your-service-app-client-secret>"

# Define the scopes your app needs.
scope = "evo.workspace evo.discovery evo.object evo.blocksync evo.file"

# Define the token endpoint URL.
url = "https://ims.bentley.com/connect/token"

# Define the parameters for the token request.
params = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": scope,
}

# Make the token request and print the access token.
response = requests.post(url, data=params)
if response.status_code != HTTPStatus.OK:
    raise RuntimeError(f"Failed to get token: {response.status_code} {response.reason}")

token = response.json()["access_token"]
print("Access token:")
print(token)

# Decode the access token to get the expiration time.
# Note: In a production application, you should verify the token signature.
decoded = jwt.decode(token, options={"verify_signature": False}, algorithms=["RS256"])
exp_timestamp = decoded["exp"]
exp_datetime = datetime.datetime.fromtimestamp(exp_timestamp, datetime.timezone.utc)
print(f"Token expires at: {exp_datetime} UTC")